<a href="https://colab.research.google.com/github/bankvis7/cp-axtra-training-lab-3/blob/main/lab4_2_run_finetune_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 4 Part 2: Run Fine-tuned Models

เนื้อหา:
- การนำโมเดลที่ Fine-tuning ด้วย LoRA ไปใช้งาน

---

## การติดตั้ง Library

In [8]:
!uv pip install -q unsloth

In [9]:
from unsloth import FastLanguageModel
from peft import PeftModel
import torch

# ขั้นตอนที่ 1: การโหลดโมเดล

In [10]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Reubencf/gemma3-goan-finetuned",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

==((====))==  Unsloth 2025.11.2: Fast Gemma3 patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


adapter_model.safetensors:   0%|          | 0.00/47.7M [00:00<?, ?B/s]

In [11]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma3ForConditionalGeneration(
      (model): Gemma3Model(
        (vision_tower): SiglipVisionModel(
          (vision_model): SiglipVisionTransformer(
            (embeddings): SiglipVisionEmbeddings(
              (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
              (position_embedding): Embedding(4096, 1152)
            )
            (encoder): SiglipEncoder(
              (layers): ModuleList(
                (0-26): 27 x SiglipEncoderLayer(
                  (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
                  (self_attn): SiglipAttention(
                    (k_proj): lora.Linear(
                      (base_layer): Linear(in_features=1152, out_features=1152, bias=True)
                      (lora_dropout): ModuleDict(
                        (default): Dropout(p=0.1, inplace=False)
                      )
                      (lora

# ขั้นตอนที่ 2: การเตรียม Prompt

In [12]:
def create_prompt(instruction, input_text=""):
    if input_text:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input_text}

### Response:
"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:
"""

# ขั้นตอนที่ 3: การเรียกใช้งานโมเดล

In [14]:
instruction = "เขียนฟังก์ชันภาษา Python เพื่อคำนวณลำดับฟีโบนัชชี"

prompt = create_prompt(instruction)
inputs = tokenizer(text=prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
  outputs = model.generate(
      **inputs,
      max_new_tokens=128,
      temperature=0.1,
      top_p=0.9,
      do_sample=True,
  )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response.split("### Response:")[1].strip())

```python
def fibonacci(n):
    """
    คำนวณลำดับฟีโบนัชชีจนถึงลำดับที่ n
    """
    if n <= 0:
        return []
    elif n == 1:
        return [0]
    else:
        list_fib = [0, 1]
        while len(list_fib) < n:
            next_fib = list_fib[-1] + list_fib[-2]
            list_fib.append(next_fib)
        return list_fib


----